In [ ]:
import os

In [ ]:
import pandas as pd
import numpy as np

from glob import glob
from tqdm import tqdm
from scipy import interpolate

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU, AveragePooling1D, GlobalAveragePooling1D

#GPU 자원이 부족한 경우 아래 코드를 이용하세요
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

import tensorflow_addons as tfa
from keras import backend as K
from tensorflow.keras import layers

C:\Users\Sunwoo\anaconda3\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.7.0 and strictly below 2.10.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.4.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [ ]:
f_list = sorted(glob("full_data/*.xlsx"))
f_list

NameError: name 'glob' is not defined

In [ ]:
pd.read_csv(f_list[0]).shape

(26496, 18)

In [ ]:
pd.read_csv(f_list[0], index_col = 0).head(4)

,rf_10184100,rf_10184110,rf_10184140,swl,inf,sfw,ecpc,tototf,tide_level,wl_1018662,fw_1018662,wl_1018680,fw_1018680,wl_1018683,fw_1018683,wl_1019630,fw_1019630
ymdhm,,,,,,,,,,,,,,,,,
2012-05-01 00:00,0.0,0.0,0.0,24.800,555.0,219.07,24.93,555.0,445.0,310.7,469.05,300.2,0.0,290.0,729.80,275.3,540.18
2012-05-01 00:10,0.0,0.0,0.0,24.794,464.6,218.86,25.15,562.9,449.0,314.7,498.00,300.2,0.0,290.0,731.48,275.3,540.18
2012-05-01 00:20,0.0,0.0,0.0,24.789,478.1,218.69,25.31,576.4,451.0,313.7,490.68,301.2,0.0,290.0,726.42,275.3,540.18
2012-05-01 00:30,0.0,0.0,0.0,24.789,464.8,218.69,25.31,563.1,452.0,311.7,476.21,301.2,0.0,290.0,726.42,276.3,552.17


In [ ]:
train_data = []
train_label = []
num = 0

window = 30

for i in f_list[:-1]:
    
    tmp = pd.read_csv(i, index_col = 0)
    tmp[["wl_1018662_1", "wl_1018680_1", "wl_1018683_1", "wl_1019630_1"]] = tmp[["wl_1018662", "wl_1018680", "wl_1018683","wl_1019630"]] - pd.concat([tmp[:1][["wl_1018662", "wl_1018680", "wl_1018683","wl_1019630"]], tmp[:-1][["wl_1018662", "wl_1018680","wl_1018683","wl_1019630"]]], axis = 0).reset_index(drop=True)
    tmp[["wl_1018662_2", "wl_1018680_2", "wl_1018683_2", "wl_1019630_2"]] = tmp[["wl_1018662", "wl_1018680", "wl_1018683","wl_1019630"]] - pd.concat([tmp[:2][["wl_1018662", "wl_1018680", "wl_1018683","wl_1019630"]], tmp[:-2][["wl_1018662", "wl_1018680","wl_1018683","wl_1019630"]]], axis = 0).reset_index(drop=True)

    tmp = tmp.replace(" ", np.nan)
    tmp = tmp.interpolate(method = 'values')
    tmp = tmp.fillna(0)

    for j in tqdm(range(len(tmp)-window)):
        train_data.append(np.array(tmp.loc[j:j + window-1,
                                           ['rf_10184100','rf_10184110','rf_10184140',"swl", "inf", "sfw", "ecpc",
                                            "tototf", "tide_level","fw_1018662", "fw_1018680","fw_1018683", "fw_1019630",
                                            "wl_1018662", "wl_1018680", "wl_1018683","wl_1019630",
                                            "wl_1018662_1", "wl_1018680_1", "wl_1018683_1", "wl_1019630_1",
                                           "wl_1018662_2", "wl_1018680_2", "wl_1018683_2", "wl_1019630_2"]]).astype(float))
        
        train_label.append(np.array(tmp.loc[j + window, ["wl_1018662", "wl_1018680",
                                                      "wl_1018683", "wl_1019630"]]).astype(float))

TypeError: Cannot cast array data from dtype('O') to dtype('float64') according to the rule 'safe'

In [ ]:
train_data = np.array(train_data)
train_label = np.array(train_label)

print(train_data.shape)
print(train_label.shape)

np.save("train_data.npy",train_data)
np.save("train_label.npy",train_label)

(264660, 30, 25)
(264660, 4)


In [ ]:
train_data = np.load("train_data.npy")
train_label = np.load("train_label.npy")

In [ ]:
def r2(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true)))
    return (1 - SS_res/(SS_tot + K.epsilon()))

def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 

def score(y_test, y_pred):
    return rmse(y_test, y_pred) / r2(y_test, y_pred)

In [ ]:
input_shape = (train_data[0].shape[0], train_data[0].shape[1])

model = Sequential()
#model.add(GRU(256, input_shape=input_shape))
#model.add(GRU(32, return_sequences=False))
#model.add(Dense(4, activation = 'relu'))

model.add(layers.Conv1D(256, (3), activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling1D((2)))
model.add(layers.Conv1D(512, (3), activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling1D((2)))
model.add(layers.Conv1D(64, (3), activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling1D((2)))
model.add(layers.Flatten())
model.add(Dense(4, activation = 'relu'))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001,
                                     beta_1=0.9,
                                     beta_2=0.999,
                                     epsilon=1e-07)

model.compile(optimizer=optimizer,loss = score, metrics=[rmse,'mae'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 28, 256)           19456     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 14, 256)           0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 12, 512)           393728    
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 6, 512)            0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 4, 64)             98368     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 2, 64)             0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 128)              

In [ ]:
model.load_weights('epoch_001')

model.fit(train_data, train_label, epochs=5000, batch_size=4096) #512
model.save_weights('epoch_001')

Epoch 1/5000
65/65 [==============================] - 6s 87ms/step - loss: 0.8058 - rmse: 0.8056 - mae: 0.6685
Epoch 2/5000
65/65 [==============================] - 6s 86ms/step - loss: 0.8049 - rmse: 0.8047 - mae: 0.6666
Epoch 3/5000
65/65 [==============================] - 6s 86ms/step - loss: 0.8090 - rmse: 0.8088 - mae: 0.6710
Epoch 4/5000
65/65 [==============================] - 6s 88ms/step - loss: 0.8035 - rmse: 0.8034 - mae: 0.6661
Epoch 5/5000
65/65 [==============================] - 6s 86ms/step - loss: 0.8032 - rmse: 0.8030 - mae: 0.6655
Epoch 6/5000
65/65 [==============================] - 6s 85ms/step - loss: 0.8013 - rmse: 0.8011 - mae: 0.6630
Epoch 7/5000
65/65 [==============================] - 6s 87ms/step - loss: 0.8035 - rmse: 0.8033 - mae: 0.6656
Epoch 8/5000
65/65 [==============================] - 6s 89ms/step - loss: 0.8047 - rmse: 0.8046 - mae: 0.6673
Epoch 9/5000
65/65 [==============================] - 6s 86ms/step - loss: 0.8043 - rmse: 0.8042 - mae: 0.6664
E

65/65 [==============================] - 6s 87ms/step - loss: 0.8073 - rmse: 0.8071 - mae: 0.6691
Epoch 75/5000
65/65 [==============================] - 6s 86ms/step - loss: 0.8040 - rmse: 0.8038 - mae: 0.6664
Epoch 76/5000
65/65 [==============================] - 6s 88ms/step - loss: 0.8047 - rmse: 0.8045 - mae: 0.6659
Epoch 77/5000
65/65 [==============================] - 6s 86ms/step - loss: 0.8039 - rmse: 0.8037 - mae: 0.6661
Epoch 78/5000
65/65 [==============================] - 6s 88ms/step - loss: 0.7940 - rmse: 0.7938 - mae: 0.6570
Epoch 79/5000
65/65 [==============================] - 6s 90ms/step - loss: 0.8089 - rmse: 0.8087 - mae: 0.6720
Epoch 80/5000
65/65 [==============================] - 6s 89ms/step - loss: 0.8010 - rmse: 0.8008 - mae: 0.6636
Epoch 81/5000
65/65 [==============================] - 6s 86ms/step - loss: 0.8025 - rmse: 0.8023 - mae: 0.6655
Epoch 82/5000
65/65 [==============================] - 6s 92ms/step - loss: 0.8067 - rmse: 0.8065 - mae: 0.6687
Epoch 

65/65 [==============================] - 6s 86ms/step - loss: 0.7935 - rmse: 0.7933 - mae: 0.6574
Epoch 219/5000
65/65 [==============================] - 6s 89ms/step - loss: 0.8091 - rmse: 0.8090 - mae: 0.6726
Epoch 220/5000
65/65 [==============================] - 6s 90ms/step - loss: 0.8034 - rmse: 0.8032 - mae: 0.6646
Epoch 221/5000
65/65 [==============================] - 6s 86ms/step - loss: 0.8017 - rmse: 0.8015 - mae: 0.6633
Epoch 222/5000
65/65 [==============================] - 6s 89ms/step - loss: 0.8010 - rmse: 0.8008 - mae: 0.6639
Epoch 223/5000
65/65 [==============================] - 6s 86ms/step - loss: 0.8016 - rmse: 0.8014 - mae: 0.6649
Epoch 224/5000
65/65 [==============================] - 6s 90ms/step - loss: 0.8007 - rmse: 0.8005 - mae: 0.6627
Epoch 225/5000
65/65 [==============================] - 6s 87ms/step - loss: 0.8015 - rmse: 0.8013 - mae: 0.6643
Epoch 226/5000
65/65 [==============================] - 6s 93ms/step - loss: 0.8011 - rmse: 0.8009 - mae: 0.665

65/65 [==============================] - 6s 90ms/step - loss: 0.8035 - rmse: 0.8034 - mae: 0.6651
Epoch 291/5000
65/65 [==============================] - 6s 86ms/step - loss: 0.8088 - rmse: 0.8086 - mae: 0.6703
Epoch 292/5000
65/65 [==============================] - 6s 86ms/step - loss: 0.7958 - rmse: 0.7956 - mae: 0.6585
Epoch 293/5000
65/65 [==============================] - 6s 90ms/step - loss: 0.8080 - rmse: 0.8078 - mae: 0.6712
Epoch 294/5000
65/65 [==============================] - 6s 86ms/step - loss: 0.8040 - rmse: 0.8038 - mae: 0.6658
Epoch 295/5000
65/65 [==============================] - 6s 90ms/step - loss: 0.8031 - rmse: 0.8029 - mae: 0.6654
Epoch 296/5000
65/65 [==============================] - 6s 93ms/step - loss: 0.8022 - rmse: 0.8020 - mae: 0.6644
Epoch 297/5000
65/65 [==============================] - 6s 89ms/step - loss: 0.8048 - rmse: 0.8046 - mae: 0.6674
Epoch 298/5000
65/65 [==============================] - 6s 90ms/step - loss: 0.8008 - rmse: 0.8006 - mae: 0.664

65/65 [==============================] - 6s 87ms/step - loss: 0.7993 - rmse: 0.7992 - mae: 0.6607
Epoch 435/5000
65/65 [==============================] - 6s 88ms/step - loss: 0.7974 - rmse: 0.7973 - mae: 0.6607
Epoch 436/5000
65/65 [==============================] - 6s 88ms/step - loss: 0.7992 - rmse: 0.7990 - mae: 0.6608
Epoch 437/5000
65/65 [==============================] - 6s 88ms/step - loss: 0.8032 - rmse: 0.8030 - mae: 0.6652
Epoch 438/5000
65/65 [==============================] - 6s 89ms/step - loss: 0.8042 - rmse: 0.8040 - mae: 0.6665
Epoch 439/5000
65/65 [==============================] - 6s 89ms/step - loss: 0.8000 - rmse: 0.7998 - mae: 0.6626
Epoch 440/5000
65/65 [==============================] - 6s 90ms/step - loss: 0.7971 - rmse: 0.7970 - mae: 0.6601
Epoch 441/5000
65/65 [==============================] - 6s 86ms/step - loss: 0.8026 - rmse: 0.8024 - mae: 0.6661
Epoch 442/5000
65/65 [==============================] - 6s 87ms/step - loss: 0.8076 - rmse: 0.8074 - mae: 0.669

...

Epoch 4425/5000
65/65 [==============================] - 6s 92ms/step - loss: 0.7957 - rmse: 0.7955 - mae: 0.6590
Epoch 4426/5000
65/65 [==============================] - 6s 90ms/step - loss: 0.7968 - rmse: 0.7966 - mae: 0.6596
Epoch 4427/5000
65/65 [==============================] - 6s 94ms/step - loss: 0.7935 - rmse: 0.7933 - mae: 0.6570
Epoch 4428/5000
65/65 [==============================] - 6s 98ms/step - loss: 0.7977 - rmse: 0.7975 - mae: 0.6595
Epoch 4429/5000
65/65 [==============================] - 6s 89ms/step - loss: 0.8005 - rmse: 0.8003 - mae: 0.6640
Epoch 4430/5000
65/65 [==============================] - 6s 92ms/step - loss: 0.8012 - rmse: 0.8010 - mae: 0.6644
Epoch 4431/5000
65/65 [==============================] - 6s 93ms/step - loss: 0.7971 - rmse: 0.7970 - mae: 0.6602
Epoch 4432/5000
65/65 [==============================] - 6s 92ms/step - loss: 0.7916 - rmse: 0.7914 - mae: 0.6553
Epoch 4433/5000
65/65 [==============================] - 6s 88ms/step - loss: 0.8019 - r

65/65 [==============================] - 6s 87ms/step - loss: 0.8020 - rmse: 0.8019 - mae: 0.6665
Epoch 4568/5000
65/65 [==============================] - 6s 90ms/step - loss: 0.7964 - rmse: 0.7962 - mae: 0.6587
Epoch 4569/5000
65/65 [==============================] - 6s 95ms/step - loss: 0.7989 - rmse: 0.7987 - mae: 0.6615
Epoch 4570/5000
65/65 [==============================] - 6s 94ms/step - loss: 0.7914 - rmse: 0.7912 - mae: 0.6553
Epoch 4571/5000
65/65 [==============================] - 6s 89ms/step - loss: 0.8011 - rmse: 0.8009 - mae: 0.6647
Epoch 4572/5000
65/65 [==============================] - 6s 90ms/step - loss: 0.8032 - rmse: 0.8030 - mae: 0.6652
Epoch 4573/5000
65/65 [==============================] - 6s 89ms/step - loss: 0.7995 - rmse: 0.7993 - mae: 0.6637
Epoch 4574/5000
65/65 [==============================] - 6s 91ms/step - loss: 0.7951 - rmse: 0.7950 - mae: 0.6584
Epoch 4575/5000
65/65 [==============================] - 6s 89ms/step - loss: 0.7947 - rmse: 0.7945 - ma

Epoch 4710/5000
65/65 [==============================] - 6s 90ms/step - loss: 0.7951 - rmse: 0.7949 - mae: 0.6578
Epoch 4711/5000
65/65 [==============================] - 6s 91ms/step - loss: 0.8019 - rmse: 0.8017 - mae: 0.6657
Epoch 4712/5000
65/65 [==============================] - 6s 93ms/step - loss: 0.7968 - rmse: 0.7966 - mae: 0.6606
Epoch 4713/5000
65/65 [==============================] - 6s 88ms/step - loss: 0.7986 - rmse: 0.7985 - mae: 0.6621
Epoch 4714/5000
65/65 [==============================] - 6s 87ms/step - loss: 0.7967 - rmse: 0.7965 - mae: 0.6598
Epoch 4715/5000
65/65 [==============================] - 6s 91ms/step - loss: 0.7974 - rmse: 0.7973 - mae: 0.6620
Epoch 4716/5000
65/65 [==============================] - 6s 92ms/step - loss: 0.7946 - rmse: 0.7944 - mae: 0.6570
Epoch 4717/5000
65/65 [==============================] - 6s 89ms/step - loss: 0.7983 - rmse: 0.7982 - mae: 0.6613
Epoch 4718/5000
65/65 [==============================] - 6s 89ms/step - loss: 0.7953 - r

65/65 [==============================] - 6s 91ms/step - loss: 0.7971 - rmse: 0.7970 - mae: 0.6599
Epoch 4854/5000
65/65 [==============================] - 6s 90ms/step - loss: 0.7918 - rmse: 0.7916 - mae: 0.6551
Epoch 4855/5000
65/65 [==============================] - 6s 98ms/step - loss: 0.7965 - rmse: 0.7964 - mae: 0.6593
Epoch 4856/5000
65/65 [==============================] - 6s 89ms/step - loss: 0.7978 - rmse: 0.7976 - mae: 0.6603
Epoch 4857/5000
65/65 [==============================] - 6s 89ms/step - loss: 0.7921 - rmse: 0.7919 - mae: 0.6559
Epoch 4858/5000
65/65 [==============================] - 6s 88ms/step - loss: 0.8056 - rmse: 0.8054 - mae: 0.6681
Epoch 4859/5000
65/65 [==============================] - 6s 90ms/step - loss: 0.7964 - rmse: 0.7963 - mae: 0.6590
Epoch 4860/5000
65/65 [==============================] - 6s 93ms/step - loss: 0.7997 - rmse: 0.7995 - mae: 0.6627
Epoch 4861/5000
65/65 [==============================] - 6s 91ms/step - loss: 0.8032 - rmse: 0.8030 - ma

Epoch 4996/5000
65/65 [==============================] - 6s 93ms/step - loss: 0.7920 - rmse: 0.7919 - mae: 0.6547
Epoch 4997/5000
65/65 [==============================] - 6s 95ms/step - loss: 0.8061 - rmse: 0.8059 - mae: 0.6703
Epoch 4998/5000
65/65 [==============================] - 6s 92ms/step - loss: 0.7948 - rmse: 0.7946 - mae: 0.6582
Epoch 4999/5000
65/65 [==============================] - 6s 93ms/step - loss: 0.7933 - rmse: 0.7931 - mae: 0.6565
Epoch 5000/5000
65/65 [==============================] - 6s 90ms/step - loss: 0.8000 - rmse: 0.7998 - mae: 0.6637


# 추론 데이터셋 만들기

In [ ]:
test_data = []

tmp = pd.read_csv(f_list[-1])
tmp = tmp.replace(" ", np.nan)
tmp = tmp.fillna(method = 'pad')
tmp = tmp.fillna(0)
tmp.index = tmp.ymdhm

tmp['wl_1018662'].iloc[-6912:] = pd.read_csv("정답_submission.csv", index_col = 'ymdhm')['wl_1018662']
tmp['wl_1018680'].iloc[-6912:] = pd.read_csv("정답_submission.csv", index_col = 'ymdhm')['wl_1018680']
tmp['wl_1018683'].iloc[-6912:] = pd.read_csv("정답_submission.csv", index_col = 'ymdhm')['wl_1018683']
tmp['wl_1019630'].iloc[-6912:] = pd.read_csv("정답_submission.csv", index_col = 'ymdhm')['wl_1019630']

tmp = tmp.reset_index(drop=True)

tmp[["wl_1018662_1", "wl_1018680_1", "wl_1018683_1", "wl_1019630_1"]] = tmp[["wl_1018662", "wl_1018680", "wl_1018683","wl_1019630"]] - pd.concat([tmp[:1][["wl_1018662", "wl_1018680", "wl_1018683","wl_1019630"]], tmp[:-1][["wl_1018662", "wl_1018680","wl_1018683","wl_1019630"]]], axis = 0).reset_index(drop=True)  
tmp[["wl_1018662_2", "wl_1018680_2", "wl_1018683_2", "wl_1019630_2"]] = tmp[["wl_1018662", "wl_1018680", "wl_1018683","wl_1019630"]] - pd.concat([tmp[:2][["wl_1018662", "wl_1018680", "wl_1018683","wl_1019630"]], tmp[:-2][["wl_1018662", "wl_1018680","wl_1018683","wl_1019630"]]], axis = 0).reset_index(drop=True)

tmp

<ipython-input-19-362c8142d941>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['wl_1018662'].iloc[-6912:] = pd.read_csv("정답_submission.csv", index_col = 'ymdhm')['wl_1018662']
<ipython-input-19-362c8142d941>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['wl_1018680'].iloc[-6912:] = pd.read_csv("정답_submission.csv", index_col = 'ymdhm')['wl_1018680']
<ipython-input-19-362c8142d941>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a

,Unnamed: 0,ymdhm,rf_10184100,rf_10184110,rf_10184140,swl,inf,sfw,ecpc,tototf,...,wl_1019630,fw_1019630,wl_1018662_1,wl_1018680_1,wl_1018683_1,wl_1019630_1,wl_1018662_2,wl_1018680_2,wl_1018683_2,wl_1019630_2
0,0,2022-05-01 00:00,0.0,0.0,0.0,25.20,739.36,218.73,25.27,124.36,...,269.3,471.08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2022-05-01 00:10,0.0,0.0,0.0,25.20,124.48,218.73,25.27,124.48,...,266.3,438.33,1.0,0.0,0.0,-3.0,0.0,0.0,0.0,0.0
2,2,2022-05-01 00:20,0.0,0.0,0.0,25.20,124.20,218.73,25.27,124.20,...,264.3,417.17,-1.0,0.0,0.0,-2.0,0.0,0.0,0.0,-5.0
3,3,2022-05-01 00:30,0.0,0.0,0.0,25.20,124.35,218.73,25.27,124.35,...,263.3,406.79,-1.0,0.0,0.0,-1.0,-2.0,0.0,0.0,-3.0
4,4,2022-05-01 00:40,0.0,0.0,0.0,25.19,0.00,218.36,25.64,124.42,...,264.3,417.17,-2.0,0.0,0.0,1.0,-3.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11371,11371,2022-07-18 23:10,0.0,0.0,0.0,25.04,259.23,212.86,31.14,259.23,...,306.3,974.40,0.0,9.0,9.0,3.0,-1.0,9.0,17.0,7.0
11372,11372,2022-07-18 23:20,0.0,0.0,0.0,25.04,260.46,212.86,31.14,260.46,...,308.3,1006.88,-1.0,8.0,5.0,2.0,-1.0,17.0,14.0,5.0
11373,11373,2022-07-18 23:30,0.0,0.0,0.0,25.04,259.37,212.86,31.14,259.37,...,310.3,1039.90,12.0,4.0,5.0,2.0,11.0,12.0,10.0,4.0
11374,11374,2022-07-18 23:40,0.0,0.0,0.0,25.04,259.13,212.86,31.14,259.13,...,312.3,1073.46,10.0,4.0,5.0,2.0,22.0,8.0,10.0,4.0


In [ ]:
for j in tqdm(range(len(tmp)-window)): 
    test_data.append(np.array(tmp.loc[j:j + window-1,
                                      ['rf_10184100','rf_10184110','rf_10184140',"swl", "inf", "sfw", "ecpc",
                                       "tototf", "tide_level","fw_1018662", "fw_1018680","fw_1018683", "fw_1019630",
                                       "wl_1018662", "wl_1018680", "wl_1018683","wl_1019630",
                                       "wl_1018662_1", "wl_1018680_1", "wl_1018683_1", "wl_1019630_1",
                                       "wl_1018662_2", "wl_1018680_2", "wl_1018683_2", "wl_1019630_2"]]).astype(float))

100%|██████████| 11346/11346 [00:05<00:00, 2237.32it/s]


In [ ]:
test_data = np.array(test_data)

print(test_data.shape)
print(test_data[0].shape)

(11346, 30, 25)
(30, 25)


## 제출 파일 만들기

In [ ]:
pred = model.predict(test_data[-6912:])
pred = pd.DataFrame(pred)

sample_submission = pd.read_csv("sample_submission.csv")

sample_submission["wl_1018662"] = pred[0]
sample_submission["wl_1018680"] = pred[1]
sample_submission["wl_1018683"] = pred[2]
sample_submission["wl_1019630"] = pred[3]

sample_submission.to_csv("baseline5.csv", index = False)

In [ ]:
base = np.array(pd.read_csv('baseline5.csv').iloc[:,1:])
base

array([[280.60437, 276.67935, 274.83374, 268.48843],
       [281.53384, 276.43796, 274.4909 , 267.14178],
       [284.02603, 276.6683 , 273.82272, 264.78757],
       ...,
       [297.25494, 306.9541 , 307.9859 , 309.17426],
       [308.227  , 310.4705 , 311.934  , 311.2069 ],
       [324.69424, 315.53885, 317.57083, 311.38913]])

In [ ]:
y = np.array(pd.read_csv('정답_submission.csv').iloc[:,1:-1])
y

array([[278.7, 276.2, 274. , 269.3],
       [280.7, 277.2, 274. , 267.3],
       [283.7, 277.2, 274. , 264.3],
       ...,
       [298.7, 307.2, 309. , 310.3],
       [308.7, 311.2, 314. , 312.3],
       [325.7, 314.2, 317. , 313.3]])